In [4]:
import pandas as pd
from  sqlalchemy import create_engine
import os

In [10]:
# Reemplaza con tus propios datos
user = 'root'
password = 'yaris'  # La contraseña que estableciste al instalar MySQL
host = '127.0.0.1'  # O 'localhost'
database = 'olist_db' # El nombre de la base de datos que creaste

# Creamos la cadena de conexión
connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"

# Creamos el motor de conexión con SQLAlchemy
engine = create_engine(connection_string)

In [8]:
# Importamos las librerías necesarias
import pandas as pd
from sqlalchemy import create_engine
import os

# --- CONFIGURA TUS DATOS AQUÍ ---
user = 'root'
password = 'yaris'  # ¡Cambia esto por tu contraseña real!
host = '127.0.0.1'                  # O 'localhost'
database = 'olist_db'               # El nombre de tu base de datos

# --- CREAMOS EL MOTOR DE CONEXIÓN ---
connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)

# Mensaje de confirmación (si no hay error, todo bien)
print("¡Motor de conexión a MySQL creado con éxito!")

¡Motor de conexión a MySQL creado con éxito!


In [10]:
# Verificamos las tablas que se han creado en MySQL
query = "SHOW TABLES;"
tables_df = pd.read_sql(query, engine)

print("\n--- Tablas creadas en la base de datos 'olist_db' ---")
print(tables_df)


--- Tablas creadas en la base de datos 'olist_db' ---
                  Tables_in_olist_db
0                          customers
1                        geolocation
2                        order_items
3                     order_payments
4                      order_reviews
5                             orders
6  product_category_name_translation
7                           products
8                            sellers


In [11]:
# Usamos pd.read_sql para ejecutar la consulta y obtener un DataFrame
df_pedidos_clientes = pd.read_sql(query, engine)

# Echamos un vistazo al resultado
df_pedidos_clientes.head()

,Tables_in_olist_db
0,customers
1,geolocation
2,order_items
3,order_payments
4,order_reviews
